In [1]:
import cv2
import numpy as np
import mediapipe as mp
from collections import deque
from PIL import Image

In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.55)
mpDraw = mp.solutions.drawing_utils

In [3]:
mp_drawing = mp.solutions.drawing_utils

In [4]:
def coordinates(lm1,lm2):
    global b

    max_bot = min([l[1] for l in lm1])
    min_top = max([l[1] for l in lm2])
    b=lm1
    if max_bot>min_top:
        max_bot=min([l[1] for l in lm2])
        min_top=max([l[1] for l in lm1])
        b=lm2
    rnge=min_top-max_bot
    c_y=rnge*.5+max_bot
    c_x = int((min([l[0] for l in b])+max([l[0] for l in b])) / 2)
    c=(c_x,c_y)
    r=int(rnge*.4)
    c_=int((min([l[0] for l in b])-max([l[0] for l in b])) / 2)
    if r>c_x:
        r=c_x
    r=r*rp
    roi=[int(c_x-r),int(c_x+r),int(c_y-r),int(c_y+r)]
    return roi

In [5]:
def coordinates_for_hold(t,k):
     
     c_x = int((min([l[0] for l in t])+max([l[0] for l in t])) / 2)
     r=k[3]-k[2]
     r=r*.5
     c_y = min([l[1] for l in t])-r*rp*.8
     
     r=r*rp
     roi=[int(c_x-r),int(c_x+r),int(c_y-r),int(c_y+r)]
     return roi
    
    

In [6]:
global al
global temp

al=0
def rasengan(lm1,lm2,f_dash,func):
 global al
 global temp
 base_img1 = f_dash

 image = Image.open('rasengan2.png')
 if al ==0:
    temp=Image.open('rasengan2.png')
    temp.save('rasengant.png')
    al=1
 
 rotated_image = image.rotate(360*rr,expand=False)
 rotated_image.save('rasengan2.png')
   
    
 overlay_img1 = cv2.imread('rasengan2.png', cv2.IMREAD_UNCHANGED)
 k=func
 base_img = base_img1[k[2]:k[3],k[0]:k[1]]
 if abs(k[1]-k[0])==0 or abs(k[3]-k[2])==0:
        return base_img
 overlay_img = cv2.resize(overlay_img1, (abs(k[1]-k[0]),abs(k[3]-k[2])))
 alpha_channel = overlay_img[:, :, 3]
 alpha_mask = cv2.merge((alpha_channel, alpha_channel, alpha_channel))
 rgb_channels = overlay_img[:, :, :3]
 masked_rgb = cv2.bitwise_and(rgb_channels, alpha_mask)
 inverted_alpha = cv2.bitwise_not(alpha_channel)
 inverted_alpha_mask = cv2.merge((inverted_alpha, inverted_alpha, inverted_alpha))
 
 if base_img.shape!=inverted_alpha_mask.shape:
    inverted_alpha_mask=base_img
 masked_base = cv2.bitwise_and(base_img, inverted_alpha_mask)
 if masked_base is None:
     masked_base=masked_rgb  
 if masked_base.shape!= masked_rgb.shape:
    masked_rgb=masked_base
 result = cv2.addWeighted(masked_base, 1, masked_rgb, 1, 0)
 
 return result

 

In [7]:
def check_hands_position(lm1,lm2):
            left_y = sum([l[1] for l in lm1]) / len(lm1)
            right_y = sum([l[1] for l in lm2]) / len(lm2)
            
            if left_y < right_y:
                min_bot = min([l[0] for l in lm1])
                max_bot = max([l[0] for l in lm1])
                min_top = min([l[0] for l in lm2])
                max_top = max([l[0] for l in lm2])
                max_l=min_bot-(max_bot-min_bot)*.3
                max_r=max_bot+(max_bot-min_bot)*.3
                if(min_top>max_l and max_top<max_r):
                    return True
            else:
                min_bot = min([l[0] for l in lm2])
                max_bot = max([l[0] for l in lm2])
                min_top = min([l[0] for l in lm1])
                max_top = max([l[0] for l in lm1])
                max_l=min_bot-(max_bot-min_bot)*.3
                max_r=max_bot+(max_bot-min_bot)*.3
                if(min_top>max_l and max_top<max_r):
                    return True
                
 
    

In [8]:
cap = cv2.VideoCapture(0)
ret = True
global rp
global rr
global time
time=30
rp=0.01
rr=0
ch=0
while ret:
    ret, frame = cap.read()
    x, y, c = frame.shape
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hnds=0
    if result.multi_hand_landmarks:
        lm1 = []
        lm2 = []
        o1=[]
        o2=[]
        for handslms in result.multi_hand_landmarks:
            hnds+=1
            for lm in handslms.landmark:            
               if hnds==1: 
                lmx = int(lm.x*1200 )
                lmy = int(lm.y*900 )
                lm1.append([lmx, lmy])

               elif hnds==2:
                lmx = int(lm.x*1200 )
                lmy = int(lm.y*900 )
                lm2.append([lmx, lmy])

        # to draw hand line
           # mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(250,250,120), thickness=2, circle_radius=2 ), mp_drawing.DrawingSpec(   color=(250,250,120),   thickness=3,   circle_radius=3 ))
    f_dash=cv2.resize(frame, (1200,900), interpolation=cv2.INTER_LINEAR)
        
    if hnds==2:
       
       if check_hands_position(lm1,lm2):
         ch=0
         k=coordinates(lm1,lm2)
         old_k=k
         kk=rasengan(lm1,lm2,f_dash,k)
         if kk.shape==f_dash[k[2]:k[3],k[0]:k[1]].shape:
            f_dash[k[2]:k[3],k[0]:k[1]]=kk 
            if rp<=.8:
               rp+=.05
            rr+=.01
       else:
         if rp>0.8:
            if ch==0:
             left_y = sum([l[1] for l in lm1])
             right_y = sum([l[1] for l in lm2])
             ch=1
             if left_y>right_y:
                t=lm2
                s=1
             else :
                t=lm1
                s=2
            if s==1:
                t=lm1
            else:
                t=lm2
            k=coordinates_for_hold(t,old_k)
            kk=rasengan(lm1,lm2,f_dash,k)
            if kk.shape==f_dash[k[2]:k[3],k[0]:k[1]].shape:
              f_dash[k[2]:k[3],k[0]:k[1]]=kk 
    
    
    
    
    elif hnds==1:   
       #if only hand with rasengan taken out rasengan appears on other hand   
        if rp>0.8:
            
            t=lm1
            k=coordinates_for_hold(t,old_k)
            if time==0:
                old_k=k
            else:
                time-=1
            kk=rasengan(lm1,lm2,f_dash,k)
            if kk.shape==f_dash[k[2]:k[3],k[0]:k[1]].shape:
              f_dash[k[2]:k[3],k[0]:k[1]]=kk 
            
    else:
        time=30
        rp=.01
        ch=0
        rr=0
               
    
            
    cv2.imshow("Output",f_dash) 
    if cv2.waitKey(1)==ord('q'):
              break

cap.release()
cv2.destroyAllWindows()
temp=Image.open('rasengant.png')
temp.save('rasengan2.png')



1.96
8.82
16.060000000000002
24.480000000000004
32.760000000000005
41.34
49.91
59.04
66.83
74.52
81.6
86.80000000000001
101.87000000000002
95.70000000000002
103.66000000000003
114.00000000000003
125.55000000000004
126.36000000000004
128.79000000000005
137.70000000000005
153.09000000000006
153.09000000000006
163.62000000000006
172.53000000000006
179.01000000000005
182.25000000000006
196.02000000000007
188.73000000000008
191.16000000000005
187.92000000000007
186.30000000000007
186.30000000000007
187.92000000000007
102.87000000000003
0.78
